In [2]:
!pip3 install tensorflow-serving-api==2.8.0 opencv-python mediapipe pygame flask

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import cv2
import mediapipe as mp
import tensorflow as tf
import numpy as np
import pygame
import random


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Initialize Pygame
pygame.init()

# Set up the screen
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
BLOCK_SIZE = 20
GRID_WIDTH = SCREEN_WIDTH // BLOCK_SIZE
GRID_HEIGHT = SCREEN_HEIGHT // BLOCK_SIZE
FPS = 10

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Snake Game")
clock = pygame.time.Clock()

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)



# Snake class
class Snake:
    gesture_labels = ['openPlam','closeFist','openPlamToLeft','openPlamToRight']
    def __init__(self):
        self.length = 1
        self.positions = [((SCREEN_WIDTH / 2), (SCREEN_HEIGHT / 2))]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])
        self.color = RED

    def get_head_position(self):
        return self.positions[0]

    def turn(self, point):
        if self.length > 1 and (point[0]*-1, point[1]*-1) == self.direction:
            return
        else:
            self.direction = point

    def move(self):
        cur = self.get_head_position()
        x, y = self.direction
        new = (((cur[0] + (x*BLOCK_SIZE)) % SCREEN_WIDTH), (cur[1] + (y*BLOCK_SIZE)) % SCREEN_HEIGHT)
        if len(self.positions) > 2 and new in self.positions[2:]:
            self.reset()
        else:
            self.positions.insert(0, new)
            if len(self.positions) > self.length:
                self.positions.pop()

    def reset(self):
        self.length = 1
        self.positions = [((SCREEN_WIDTH / 2), (SCREEN_HEIGHT / 2))]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])

    def draw(self, surface):
        for p in self.positions:
            r = pygame.Rect((p[0], p[1]), (BLOCK_SIZE, BLOCK_SIZE))
            pygame.draw.rect(surface, self.color, r)
            pygame.draw.rect(surface, WHITE, r, 1)

#     def handle_keys(self):
#         for event in pygame.event.get():
#             if event.type == pygame.QUIT:
#                 pygame.quit()
#             elif event.type == pygame.KEYDOWN:
#                 if event.key == pygame.K_UP:
#                     self.turn(UP)
#                 elif event.key == pygame.K_DOWN:
#                     self.turn(DOWN)
#                 elif event.key == pygame.K_LEFT:
#                     self.turn(LEFT)
#                 elif event.key == pygame.K_RIGHT:
#                     self.turn(RIGHT)
                    
    def handle_keys(self,handGesture):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            elif event.type == pygame.KEYDOWN:
                if  gesture_label[0] == handGesture:
                    self.turn(UP)
                elif gesture_label[1] == handGesture:
                    self.turn(DOWN)
                elif gesture_label[2] == handGesture:
                    self.turn(LEFT)
                elif gesture_label[3] == handGesture:
                    self.turn(RIGHT)

# Gesture recognition setup
mp_holistic = mp.solutions.holistic # Holistic model

# TensorFlow model for gesture recognition
# Replace this with your trained model
model = tf.keras.models.load_model('action.hdf5')

# # Define gesture labels
# gesture_labels = {0: 'openPlam', 1: 'closeFist',2: 'openPlamToLeft',3:'openPlamToRight'}

# Function to preprocess the hand image
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# Gesture recognition function
def recognize_gesture(results):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for point in hand_landmarks.landmark:
                landmarks.append([point.x, point.y, point.z])
            landmarks = np.array(landmarks, dtype=np.float32).reshape(1, -1)
            prediction = model.predict(landmarks)
            label = gesture_labels[np.argmax(prediction)]
            return label
    return None

# Direction constants
UP = (0, -1)
DOWN = (0, 1)
LEFT = (-1, 0)
RIGHT = (1, 0)

# Main function
def main():
    snake = Snake()
    food = pygame.Rect(random.randint(0, GRID_WIDTH - 1) * BLOCK_SIZE,
                       random.randint(0, GRID_HEIGHT - 1) * BLOCK_SIZE,
                       BLOCK_SIZE, BLOCK_SIZE)
    
    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Process frame for gesture recognition
            image, results = mediapipe_detection(frame, holistic)
            gesture_label = recognize_gesture(results)

            # Integrate gesture recognition with the game
            if gesture_label == 'openPlam':
                snake.handle_keys(gesture_label)
            elif gesture_label =='closeFist':
                snake.handle_keys(gesture_label)
            elif gesture_label == 'openPlamToLeft':
                snake.handle_keys(gesture_label)
            elif gesture_label == 'openPlamToRight':
                snake.handle_keys(gesture_label)

            # Draw the snake and food on the Pygame screen
            screen.fill(BLACK)
            snake.move()
            snake.draw(screen)
            pygame.draw.rect(screen, WHITE, food)
            pygame.display.update()
            clock.tick(FPS)

            # Check if snake ate the food
            if snake.get_head_position() == food.topleft:
                snake.length += 1
                food = pygame.Rect(random.randint(0, GRID_WIDTH - 1) * BLOCK_SIZE,
                                   random.randint(0, GRID_HEIGHT - 1) * BLOCK_SIZE,
                                   BLOCK_SIZE, BLOCK_SIZE)

            # Check for Pygame events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


AttributeError: type object 'SolutionOutputs' has no attribute 'multi_hand_landmarks'

In [ ]:
import cv2
import mediapipe as mp
import tensorflow as tf
import numpy as np
import pygame
import random

# Initialize Pygame
pygame.init()

# Set up the screen
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
BLOCK_SIZE = 20
GRID_WIDTH = SCREEN_WIDTH // BLOCK_SIZE
GRID_HEIGHT = SCREEN_HEIGHT // BLOCK_SIZE
FPS = 10

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Snake Game")
clock = pygame.time.Clock()

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# Snake class
class Snake:
    def __init__(self):
        self.length = 1
        self.positions = [((SCREEN_WIDTH / 2), (SCREEN_HEIGHT / 2))]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])
        self.color = RED

    def get_head_position(self):
        return self.positions[0]

    def turn(self, point):
        if self.length > 1 and (point[0]*-1, point[1]*-1) == self.direction:
            return
        else:
            self.direction = point

    def move(self):
        cur = self.get_head_position()
        x, y = self.direction
        new = (((cur[0] + (x*BLOCK_SIZE)) % SCREEN_WIDTH), (cur[1] + (y*BLOCK_SIZE)) % SCREEN_HEIGHT)
        if len(self.positions) > 2 and new in self.positions[2:]:
            self.reset()
        else:
            self.positions.insert(0, new)
            if len(self.positions) > self.length:
                self.positions.pop()

    def reset(self):
        self.length = 1
        self.positions = [((SCREEN_WIDTH / 2), (SCREEN_HEIGHT / 2))]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])

    def draw(self, surface):
        for p in self.positions:
            r = pygame.Rect((p[0], p[1]), (BLOCK_SIZE, BLOCK_SIZE))
            pygame.draw.rect(surface, self.color, r)
            pygame.draw.rect(surface, WHITE, r, 1)

    def handle_keys(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    self.turn(UP)
                elif event.key == pygame.K_DOWN:
                    self.turn(DOWN)
                elif event.key == pygame.K_LEFT:
                    self.turn(LEFT)
                elif event.key == pygame.K_RIGHT:
                    self.turn(RIGHT)

# Gesture recognition setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# TensorFlow model for gesture recognition
# Replace this with your trained model
model = tf.keras.models.load_model('action.hdf5')

# Define gesture labels
gesture_labels = {0: 'Fist', 1: 'Open'}

# Function to preprocess the hand image
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return results, image

# Gesture recognition function
def recognize_gesture(results):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for point in hand_landmarks.landmark:
                landmarks.append([point.x, point.y, point.z])
            landmarks = np.array(landmarks, dtype=np.float32).reshape(1, -1)
            prediction = model.predict(landmarks)
            label = gesture_labels[np.argmax(prediction)]
            return label
    return None

# Direction constants
UP = (0, -1)
DOWN = (0, 1)
LEFT = (-1, 0)
RIGHT = (1, 0)

# Main function
def main():
    snake = Snake()
    food = pygame.Rect(random.randint(0, GRID_WIDTH - 1) * BLOCK_SIZE,
                       random.randint(0, GRID_HEIGHT - 1) * BLOCK_SIZE,
                       BLOCK_SIZE, BLOCK_SIZE)
    while True:
        screen.fill(BLACK)
        snake.handle_keys()
        snake.move()
        snake.draw(screen)

        pygame.draw.rect(screen, WHITE, food)
        pygame.display.update()
        clock.tick(FPS)

        # Gesture recognition
        ret, frame = cap.read()
        results, _ = preprocess_image(frame)
        gesture_label = recognize_gesture(results)
        
        if gesture_label == 'Fist':
            snake.turn(UP)
        elif gesture_label == 'Open':
            snake.turn(DOWN)

        if snake.get_head_position() == food.topleft:
            snake.length += 1
            food = pygame.Rect(random.randint(0, GRID_WIDTH - 1) * BLOCK_SIZE,
                               random.randint(0, GRID_HEIGHT - 1) * BLOCK_SIZE,
                               BLOCK_SIZE, BLOCK_SIZE)

if __name__ == "__main__":
    main()
